In [1]:
import torch
import torch.optim as optim
import pandas as pd
import numpy as np
from physics import combined_loss, Emb, ANN, Param

In [2]:
train_df = pd.read_csv('train_data.csv')

In [3]:
def prepare_batch(df):
    t = torch.tensor(df['t_normalized'].values, dtype=torch.float32).unsqueeze(1)
    t.requires_grad_(True)  

    T_C_norm = torch.tensor(df['T_celsius_norm'].values, dtype=torch.float32).unsqueeze(1)
    T_C = torch.tensor(df['T_celsius'].values, dtype=torch.float32).unsqueeze(1)
    I = torch.tensor(df['I_amperes_norm'].values, dtype=torch.float32).unsqueeze(1)
    bat_idx = torch.tensor(df['battery_idx'].values, dtype=torch.long)
    C_target = torch.tensor(df['C_target'].values, dtype=torch.float32).unsqueeze(1)
    
    return t, T_C, T_C_norm, I, bat_idx, C_target

In [4]:
# Training parameters
learning_rate = 0.001
num_epochs = 1000
lambda_phys = 1.0  
lambda_data = 1.0
warmup_epochs = 200 

all_params = list(Emb.parameters()) + list(ANN.parameters()) + list(Param.parameters())
optimizer = optim.Adam(all_params, lr=learning_rate)

In [5]:
for epoch in range(num_epochs):
    if epoch == warmup_epochs:
        print("Switching to L-BFGS optimizer...")
        optimizer = optim.LBFGS(all_params, lr=0.1, max_iter=20, history_size=10)
    
    # Prepare batch
    t, T_C, T_C_norm, I, bat_idx, C_target = prepare_batch(train_df)
    
    if epoch < warmup_epochs:
        # Adam training
        optimizer.zero_grad()
        total_loss, params, physics_loss, data_loss, rhs, dCdt, arrhenius_term, I_mag = combined_loss(
            t, T_C, T_C_norm, I, bat_idx, C_target, lambda_phys, lambda_data, epoch
        )
        total_loss.backward()
        optimizer.step()
    else:
        # L-BFGS training (requires closure function)
        def closure():
            optimizer.zero_grad()
            total_loss, params, physics_loss, data_loss, rhs, dCdt, arrhenius_term, I_mag = combined_loss(
                t, T_C, T_C_norm, I, bat_idx, C_target, lambda_phys, lambda_data, epoch
            )
            total_loss.backward()
            return total_loss
        
        total_loss = optimizer.step(closure)
        
        # Get current losses for logging
        with torch.no_grad():
            t_eval, T_C_eval, T_C_norm_eval, I_eval, bat_idx_eval, C_target_eval = prepare_batch(train_df)
            t_eval.requires_grad_(True)  # Re-enable gradients for physics computation

        total_loss, params, physics_loss, data_loss, rhs, dCdt, arrhenius_term, I_mag = combined_loss(
            t_eval, T_C_eval, T_C_norm_eval, I_eval, bat_idx_eval, C_target_eval, lambda_phys, lambda_data, epoch
        )
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Total Loss = {total_loss:.6f}, "
              f"Physics Loss = {physics_loss:.6f}, Data Loss = {data_loss:.6f}, "
              f"RHS = {rhs.mean().item():.6f}, dCdt = {dCdt.mean().item():.6f}, "
              f"Arrhenius Term = {arrhenius_term.mean().item():.6f}, I_mag = {I_mag.mean().item():.6f}")


Epoch 0: Total Loss = 5.550944, Physics Loss = 0.999983, Data Loss = 2.775472, RHS = -0.000000, dCdt = 0.117500, Arrhenius Term = 0.000000, I_mag = 0.340005
Epoch 100: Total Loss = 0.142567, Physics Loss = 0.999994, Data Loss = 0.071283, RHS = -0.000000, dCdt = -0.351211, Arrhenius Term = 0.000000, I_mag = 0.340005
Switching to L-BFGS optimizer...
Epoch 200: Total Loss = 0.002584, Physics Loss = 0.999996, Data Loss = 0.001292, RHS = -0.000000, dCdt = -0.488614, Arrhenius Term = 0.000000, I_mag = 0.340005
Epoch 300: Total Loss = 0.000335, Physics Loss = 0.999971, Data Loss = 0.000167, RHS = -0.000000, dCdt = -0.312709, Arrhenius Term = 0.000000, I_mag = 0.340005
Epoch 400: Total Loss = 0.000335, Physics Loss = 0.999973, Data Loss = 0.000167, RHS = -0.000000, dCdt = -0.313182, Arrhenius Term = 0.000000, I_mag = 0.340005
Epoch 500: Total Loss = 0.000335, Physics Loss = 0.999974, Data Loss = 0.000167, RHS = -0.000000, dCdt = -0.313424, Arrhenius Term = 0.000000, I_mag = 0.340005
Epoch 600:

In [ ]:
import torch
from sklearn.metrics import r2_score
import numpy as np

test_data = pd.read_csv('test_data.csv')

def evaluate_model_normalized(test_df):
    with torch.no_grad():
        t, T_C, T_C_norm, I, bat_idx, C_target = prepare_batch(test_df)
        
        # Get embeddings and predictions
        emb = Emb(bat_idx)
        C_pred = ANN(t, T_C_norm, I, emb)
        
        # Convert to numpy for easier computation
        y_true = C_target.numpy().flatten()
        y_pred = C_pred.numpy().flatten()
        
        # Compute metrics
        mse = torch.nn.MSELoss()(C_pred, C_target).item()
        mae = torch.nn.L1Loss()(C_pred, C_target).item()
        
        # R² Score (coefficient of determination) - best metric for regression
        r2 = r2_score(y_true, y_pred)
        
        # Normalized RMSE (NRMSE) - RMSE normalized by mean target value
        rmse = np.sqrt(mse)
        nrmse = rmse / np.mean(y_true)
        
        # Mean Absolute Percentage Error (MAPE) - percentage-based error
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        
        # Max error - worst single prediction
        max_error = np.max(np.abs(y_true - y_pred))
        
        return {
            'MSE': mse,
            'MAE': mae,
            'R2': r2,                    
            'NRMSE': nrmse,              
            'MAPE': mape,                
            'Max_Error': max_error,     
            'predictions': y_pred,
            'targets': y_true
        }

# Test the model with comprehensive metrics
test_results = evaluate_model_normalized(test_data)
print(f"Test MSE: {test_results['MSE']:.6f}")
print(f"Test MAE: {test_results['MAE']:.6f}")
print(f"Test R²: {test_results['R2']:.6f}")
print(f"Test NRMSE: {test_results['NRMSE']:.6f} ({test_results['NRMSE']*100:.2f}%)")
print(f"Test MAPE: {test_results['MAPE']:.2f}%")
print(f"Test Max Error: {test_results['Max_Error']:.6f}")


Test MSE: 0.000098
Test MAE: 0.007406
Test R²: 0.997463
Test NRMSE: 0.006327 (0.63%)
Test MAPE: 0.46%
Test Max Error: 0.031207


In [8]:
# In a new cell at the end of Train.ipynb
torch.save(ANN.state_dict(), 'ann_model.pth')
torch.save(Emb.state_dict(), 'emb_model.pth')
torch.save(Param.state_dict(), 'param_model.pth')

print("Models saved successfully!")

Models saved successfully!
